In [1]:
import pandas as pd
import pyarrow
import urllib.request as urllib2
import os
import json 
import pickle
import csv

import re
import numpy as np
from pathlib import Path
import tqdm


In [8]:
files = sorted(Path('.\data').glob('*.pickle'))
data_dict = {}
for fn in files:
    with open(fn, 'rb') as f:
        part = pickle.load(f)
    data_dict.update(part)

len(data_dict)

29860

In [23]:
data = pd.DataFrame(columns = ['Name', 'paragraph', 'Species', 'Neighbours'])
i=0
for key, value in data_dict.items():
    if value['is_character']:
        data.loc[i, 'Name'] = value['title']
        data.loc[i, 'paragraph'] = value['paragraph']
        if 'Physical description' in value['side_bar']:
            species = value['side_bar']['Physical description']['Species']
        elif 'Familial information' in value['side_bar']:
            species = value['side_bar']['Familial information']['Species']
        data.loc[i, 'Species'] = species
        data.loc[i, 'Neighbours'] = list(value['crosslinks'])
        i +=1
with open('./data/data_all.pickle', 'wb') as f:
    pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

In [54]:
data_characters = pd.read_parquet('StarWars_Characters.parquet', engine='pyarrow')
data_characters = data_characters.rename(columns = {'name': 'CharacterName', 'key': 'WikiLink'})
data_characters.head(5)

,CharacterName,WikiLink,url,description,species_2nd,species_3rd,species,home_world,gender,height,eye_color,skin_color,hair_color,weight
0,1138 (First Order),1138_(First_Order),https://starwars.fandom.com/wiki/1138_(First_O...,1138 was a human male stormtrooper who served ...,None,None,Human,None,Male,NaN,None,None,None,None
1,1151,1151,https://starwars.fandom.com/wiki/1151,1151 was a clone cadet who was a part of the C...,None,None,Human,Kamino,Male,NaN,Brown,Tan,Black,None
2,1174,1174,https://starwars.fandom.com/wiki/1174,"1174, nicknamed ""Whiplash,"" was a clone troope...",None,None,Human,Kamino,Male,NaN,Brown,Tan,Black,None
3,224 (Coruscant Security Force),224_(Coruscant_Security_Force),https://starwars.fandom.com/wiki/224_(Coruscan...,224 was the designation of a human male office...,None,None,Human,None,Male,NaN,None,Light,None,None
4,3-6,3-6,https://starwars.fandom.com/wiki/3-6,3-6 was a male death trooper in DT-F16's squad...,None,None,Human,None,Male,NaN,None,None,None,None


In [23]:
data_text = pd.read_parquet('StarWars_Descriptions.parquet', engine='pyarrow')
data_text.head()

,key,title,is_character,description
0,Blind_Alley_techniques,"""Blind Alley"" techniques",False,"""Blind Alley"" techniques were a form of hand-t..."
1,Blinder_4B2_jamming_array,"""Blinder"" 4B2 jamming array",False,"The ""Blinder"" 4B2 jamming array was a type of ..."
2,Blissex-head_bit-driver,"""Blissex-head"" bit-driver",False,"""Blissex-head"" bit-drivers were a type of Pile..."
3,Bunker_Buster_shell,"""Bunker Buster"" shell",False,"""Bunker Buster"" shells were missiles used by b..."
4,Changeling_Mark_71NB,"""Changeling"" Mark 71NB",False,"The ""Changeling"" Mark 71NB was a type of racin..."
